<a href="https://colab.research.google.com/github/topperf15/petrinets/blob/main/sortie_rate_gen_mod_pn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sortie Rate Generation Model Based on Petri Nets, using the pnet.py module developed separately and available from COPADS.

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 17 17:34:21 2024

@author: andrew_s
"""

import sys
sys.path.append('/Users/andrew_s/Desktop')

import pnet as pn
import random
# time steps are one hour
vul = 6

net = pn.PNet()
# The starting conditions
net.add_places('hgr', {'ac': 4})
net.add_places('FL', {'ac': 4})
net.add_places('air', {'ac': 0, 'pilot': 0})
net.add_places('lost', {'ac': 0})
net.add_places('downed', {'pilot': 0})
net.add_places('readyrm', {'pilot': 8})
net.add_places('rest', {'pilot': 2})

net.add_rules('land', 'incubate', ['6; air.ac -> hgr.ac; air.ac >0'])
net.add_rules('rest', 'incubate', ['6; air.pilot -> rest.pilot; air.pilot >0'])
net.add_rules('ready', 'incubate', ['3; hgr.ac -> FL.ac; hgr.ac > 0'])
net.add_rules('wait', 'incubate', ['8; rest.pilot -> readyrm.pilot; rest.pilot >0'])

def launch_sortie(places):
    place = places['air']
    ac = place.attributes['ac']
    if ac<4: return 4-ac
    else: return 0

net.add_rules('launch', 'function', ['FL.ac -> air.ac', launch_sortie, 'FL.ac>0; readyrm.pilot>0'])

def launch_pilot(places):
    place = places['air']
    ac = place.attributes['ac']
    pilot = place.attributes['pilot']
    return ac-pilot

net.add_rules('go_fly', 'function', ['readyrm.pilot -> air.pilot', launch_pilot, 'air.ac>0; readyrm.pilot>0'])

def loose_sortie(places):
    place = places['air']
    ac = place.attributes['ac']
    num = random.random()
    Ps = .92
    if num >= Ps and ac>1: return 1
    else: return 0

net.add_rules('lose', 'function', ['air.ac -> lost.ac', loose_sortie, 'air.ac>0'])

def loose_pilot(places):
    lostac = places['lost']
    ac = lostac.attributes['ac']
    downedac = places['downed']
    pilot = downedac.attributes['pilot']
    return ac-pilot

net.add_rules('no_fly', 'function', ['air.pilot -> downed.pilot', loose_pilot, 'air.ac>0; air.pilot>0'])

net.simulate(24, 1, 1)
# Generate results file
data, edges = net.report_tokens()
headers = ['timestep'] + data[0][1]
f = open('flight.csv', 'w')
f.write(','.join(headers) + '\n')
for tdata in data:
    tdata = [tdata[0]] + [str(x) for x in tdata[2]]
    f.write(','.join(tdata) + '\n')
f.close()

# importing networkx
import networkx as nx
# importing matplotlib.pyplot
import matplotlib.pyplot as plt

g = nx.DiGraph()
#g.add_nodes_from([d['movement'][0][0],d['movement'][1][0]])
g.add_edges_from(edges)
pos = nx.kamada_kawai_layout(g)
nx.draw_networkx(g,pos,arrows=True,with_labels=True)
plt.savefig("filename.png")

ModuleNotFoundError: No module named 'pnet'